In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/real-estate-price-prediction/Real estate.csv')
df.head()

In [ ]:
df=df.drop(['No','X1 transaction date'],axis=1)
df.head()

In [ ]:
#change to name of columns:
df.rename({'X2 house age':'house age','X3 distance to the nearest MRT station':'station',
          'X4 number of convenience stores':'stores','X5 latitude':'latidude','X6 longitude':'longitude',
          'Y house price of unit area':'price'},axis='columns',inplace=True)

In [ ]:
df.head(10)

In [ ]:
#this dataser 414 observation units and 6 columns
df.shape

In [ ]:
df.info()

In [ ]:
#have the dataset null values any observation units?
df.isnull().sum().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#corelation of columns:
plt.figure(figsize=(15,7))
sns.heatmap(df.corr(),annot=True,fmt=".3f",linewidths = .5);

In [ ]:
#descriptive statistics of columns
plt.figure(figsize=(17,7))
sns.heatmap(df.describe().T,annot=True,fmt='.2f',linewidths = .5)
plt.show()

In [ ]:
df.head()

In [ ]:
#add independent variables to x 
#add dependent variable to y
x=df.drop(['price'],axis=1)   #ı erased dependent variable by .drop funciton,on to exist dependent variable in x valuable
y=df['price']    #this is dependent variable in y valuable

In [ ]:
#independent variables
x.head()

In [ ]:
#dependent variable
y[0:5]

In [ ]:
#now ı will train-test split by function of train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
#ready:%90 for train,%10 for test 
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                              test_size=0.10,
                                              random_state=42)

In [ ]:
print('x_train_shape',x_train.shape)
print('x_test_shape',x_test.shape)
print('y_train_shape',y_train.shape)
print('y_test_shape',y_test.shape)

## train-test resmi eklenecek

In [ ]:
#now,ı import RandomForestRegressor function:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model=RandomForestRegressor(random_state=42)  #model object
rf_model.fit(x_train,y_train)   #model was fitted

In [ ]:
#predict:
rf_model.predict(x_test)

In [ ]:
y_pred=rf_model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(r2_score(y_test,y_pred))

## hyperparamether of RandomForestRegressor:

* n_estimator:number of estimators,number of trees to fit
* max_features:the maximum number of variables required for the division operation
* max_depth:depth level of each trees
* min_samples_split:expresses the minimum number of observations in that node before a node is split
* min_samples_leaf:minimum number of observations in a node

## Model Tuning:

In [ ]:
rf_params={'max_depth':list(range(1,10)),
           'max_features':[3,5,10,15],
           'n_estimators':[100,200,500,1000,2000]}

In [ ]:
rf_model=RandomForestRegressor(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rf_cv_model=GridSearchCV(rf_model,
                         rf_params,
                         cv=10,
                         n_jobs=-1).fit(x_train,y_train)

In [ ]:
#optimum parameter:
rf_cv_model.best_params_

In [ ]:
#now,ı will buil final model,to use best_params:
rf_tuned=RandomForestRegressor(max_depth=rf_cv_model.best_params_['max_depth'],
                              max_features=rf_cv_model.best_params_['max_features'],
                              n_estimators=rf_cv_model.best_params_['n_estimators']).fit(x_train,y_train)

In [ ]:
#predict:
rf_tuned.predict(x_test)

In [ ]:
y_pred=rf_tuned.predict(x_test)

In [ ]:
#test error and r*2 score:
print('test error:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('r*2 score:',r2_score(y_test,y_pred))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#let's see importance of variables:
#significance according to dependent variable:
Importance = pd.DataFrame({"Importance": rf_tuned.feature_importances_*100},
                          index = x_train.columns)

In [ ]:
Importance

In [ ]:
Importance.sort_values(by = "Importance",
axis = 0,
ascending = True).plot(kind ="barh", color = "r")
plt.xlabel("significance of independent variables")
plt.show()